In [ ]:
pip install requests
pip install bs4
pip install urllib3
pip install qrcode


In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib3
import os
import qrcode

# Função que gera um png com um QRCode para um url

def gera_qrcode(url, nome_ficheiro):

    # Criar instância de QRcode
    qr = qrcode.QRCode(
        version=1, 
        error_correction=qrcode.constants.ERROR_CORRECT_H, 
        box_size=10, 
        border=4
    )

    # Adicionar os dados ao QRcode
    qr.add_data(url)
    qr.make(fit=True)

    # Criar imagem do QRcode
    qr_img = qr.make_image(fill_color="black", back_color="white")

    # Grava a imagem
    qr_img.save(os.path.join('qrcodes', nome_ficheiro))

# Desativa alerta de verificação SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


def extrai_info_de_url(url):
    
    response = requests.get(url, verify=False)

    if response.status_code == 200:

        if 'qrcodes' not in os.listdir():
            os.mkdir('qrcodes')

        # Parse do conteúdo HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Obter todas as disciplinas da tabela, os elementos <tr> dentro de <table>
        table = soup.find('table')
        rows = table.find_all('tr')

        # Extrair da tabela cada linha <tr> com dados <td> de cada disciplina 
        for row in rows:
            columns = row.find_all('td')

            if columns:
                nome_ficheiro = columns[0].get_text(strip=True)
                url_para_qrcode = columns[1].get_text(strip=True)
                
                gera_qrcode(url_para_qrcode, nome_ficheiro + '.png')

    else:
        print('Falha ao recuperar a página.')

In [ ]:
url = 'https://deisi.ulusofona.pt/inqueritos/admin/disc'

extrai_info_de_url(url)